In [ ]:
import pandas as pd; import numpy as np
import matplotlib.pyplot as plt; import matplotlib.cm as cm
import os
from matplotlib.legend import Legend, Line2D

plt.rcParams.update({'font.size': 14}) # Set a good font size


In [ ]:
head = ['mergertime', 'simtime', 'm1', 'm2', 'e', 'Mcl',
        'mergetype', 'tlbform', 'rh', 'vk', 'vesc', 'chi_f',
        's1', 's2', 'Z', 'a', 'genmerge', 'mbh']


files = os.listdir('runs') # Find the data files

data = pd.read_csv(os.path.join('runs', files[0]), delimiter='\t', names=head)

# Grab the cluster masses and metallicities and find the unique values
Mcls = data['Mcl'].values
metals = data['Z'].values

Mcluni = np.unique(Mcls); metaluni = np.unique(metals).round(5)
print(metaluni)

metalUnitest = np.array([metals[0],0.001,0.01])

# Oldest time
tlb = 13315470519.21035

In [ ]:
data.loc[np.isin(data['Z'].values.round(5), metalUnitest)]

data.loc[2977, 'tlbform']

Here we split only by cluster mass and so we then find the total number of mergers for a given cluster over the whole range of metallicities and birth time

In [ ]:
files = os.listdir('runs') # Find the data files

data = [pd.read_csv(os.path.join('runs', file), delimiter='\t', names=head) for file in files] # Load all files

Mcl = {int(mcl): [] for mcl in Mcluni} # dictionary for mass values

# Loop through and find total number of mergers for each
for d in data:
    # Unique redshift times
    for mcl in Mcl.keys():
        # Only megrers in this cluster within Ht
        tmp = d.loc[(d['Mcl']==mcl)&(d['mergertime']<=14e9)&(d['simtime']<=1e9)]
        # Find ejected and incluster mergers
        if len(tmp)!=0:
            ejec = np.sum(tmp['mergetype']=='ejected')
            incl = np.sum(tmp['mergetype']!='ejected')

            Mcl[mcl].append([ejec, incl, len(tmp)])

for mcl in Mcl.keys(): Mcl[mcl] = np.asarray(Mcl[mcl])

fig, ax = plt.subplots(figsize=(8,6))

for i, (key, item) in enumerate(Mcl.items()):

    # Plot mean and errors
    ax.errorbar(key, np.mean(item[:,1]/item[:,-1]), np.std(item[:,1]/item[:,-1]), marker='x', markersize=10, label=key, color='C'+str(i))
    # ax.scatter(key, np.mean(item[:,-1]), marker='x', color='black')

####### PETAR DATA ##########
petar = [np.array([[0,2,2],[0,1,1],[0,1,1]]), np.array([[1,2,3],[0,1,1],[0,3,3]]), np.array([[1,6,7],[0,7,7],[0,6,6]])]

for i, p in enumerate(petar):
    ax.scatter(Mcluni[i], sum(p[:,1])/sum(p[:,-1]), color='C'+str(i))
    # ax.scatter(Mcluni[::-1][i], sum(p[:,-1]), color='black')
ax.set_xlabel('Mcl')
ax.set_ylabel('$N_{\mathrm{incl}}/N_{\mathrm{tot}}$')

hand = [Line2D([0,0],[0,0], color='tab:blue', marker='o', markersize=5, lw=0),
        Line2D([0,0],[0,0], color='tab:blue', marker='x', markersize=10, lw=0)]
leg = Legend(ax, hand, labels=['PeTar Data', 'cBHBd'], ncols=2, loc='upper center')
ax.add_artist(leg)
# ax.set_yscale('log')

# ax.legend()


The above plot shows the PeTar data compared to the cBHBd data.

* Specifically the cBHBd data is run for the same cluster masses and across a grid of metallicities/redshifts. The PeTar data is only across metallicities [Z=0.01 -> Z=0.0001] and does not account for redshifts.
* The Error bars on the cBHBd data comes from running many simulations of the same clusters over the same grids, ~140 different seeds.



Below we split it up into every individual cluster (sectioning by metallicity, birth time and cluster mass)

In [ ]:
files = os.listdir('runs') # Find the data files

data = [pd.read_csv(os.path.join('runs', file), delimiter='\t', names=head) for file in files] # Load all files

Mcl = {int(mcl): [] for mcl in Mcluni} # dictionary for mass values

# Loop through and find total number of mergers for each
for d in data:
    # Unique redshift times
    tlbuni = np.unique(d['tlbform'].values)
    for mcl in Mcl.keys():
        for metal in metaluni:
            for tlb in tlbuni:
                # Only megrers in this cluster within Ht
                tmp = d.loc[(d['Mcl']==mcl)&(d['Z']==metal)&(d['tlbform']==tlb)&(d['mergertime']<=14e9)]

                # Find ejected and incluster mergers
                if len(tmp)!=0:
                    ejec = sum(tmp['mergetype']=='ejected')/len(tmp)
                    incl = sum(tmp['mergetype']!='ejected')/len(tmp)

                    Mcl[mcl].append([ejec, incl, len(tmp)])

for mcl in Mcl.keys(): Mcl[mcl] = np.asarray(Mcl[mcl])

fig, ax = plt.subplots(figsize=(8,6))

for key, item in Mcl.items():
    # Plot mean and errors
    ax.errorbar(np.mean(item[:,0]), np.mean(item[:,1]), 
                np.std(item[:,0]), np.std(item[:,1]), marker='x', markersize=10, label=key)

ax.set_xlabel('$N_{\mathrm{ej}}/N_{\mathrm{tot}}$')
ax.set_ylabel('$N_{\mathrm{incl}}/N_{\mathrm{tot}}$')

ax.legend()


Above plot is across every metallicity bin

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

norm = plt.Normalize(min(metaluni), max(metaluni))

for mcl in Mcluni:
    # print(mcl)
    for metal in metaluni:
        tmp = data.loc[(data['Mcl']==mcl)&(data['Z']==metal)&(data['mergertime']<=14e9)]    

        # print(len(tmp))
        color=cm.viridis(norm(metal))
        try:
            ax.scatter(mcl, sum(tmp['mergetype']=='ejected')/len(tmp), color=color)
        except:
            continue

fig.colorbar(cm.ScalarMappable(norm=norm, cmap='viridis'))

ax.set_xlabel('Cluster mass [$M_{cl}$]')
ax.set_ylabel('$N_{ej}/N_{tot}$')


print(max(data['simtime'])/1e6)